In [3]:
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import jieba
import re
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

# 1.准备数据 

In [18]:
# 构造数据集
class MyDataset(Dataset):
    def __init__(self, file_path, stopwords, sample=None):
        df = pd.read_csv(file_path).dropna().reset_index(drop=True)
        
        if sample:
            df = df.sample(sample).reset_index(drop=True)
            
        counter = Counter()
        sentences = []
        
        for title in tqdm(df['title']):   
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)  # 去除标点符号
            tokens = [token for token in jieba.cut(title.strip()) if token not in stopwords]
            counter.update(tokens)
            sentences.append(tokens)
        self.vocab = torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>'])
        
        # 构造输入和输出，输入是每三个词，输出是这三个词的下一个词，也就是简单的n-gram语言模型（n=3）
        self.inputs = []
        self.labels = []
        for sen in sentences:
            for i in range(len(sen) - 3):
                self.inputs.append(self.vocab.lookup_indices(sen[i: i + 3]))
                self.labels.append([self.vocab[sen[i + 3]]])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # 返回一个x和一个y
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])

In [19]:
file_path = '../../datasets/THUCNews/train.csv'
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]

dataset = MyDataset(file_path, stopwords, sample=10000)
dataloader = DataLoader(dataset=dataset, batch_size=128, shuffle=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

# 2.构建模型

In [20]:
class NNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, n_step):
        super().__init__()
        self.embed_size = embed_size
        self.n_step = n_step
        # vocab size投影到到embed size的空间中
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 构造一个隐藏层，输入大小为 步长 * embed size，输入大小为hidden_size
        # 将hidden_size投影回vocab size大小
        self.fc = nn.Sequential(
            nn.Linear(n_step * embed_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, vocab_size)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = x.reshape(-1, self.n_step * self.embed_size)
        y = self.fc(x)
        return y

In [23]:
# 初始化模型
model = NNLM(vocab_size=len(dataset.vocab), embed_size=256, hidden_size=256, n_step=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 查看模型
print(model)

NNLM(
  (embedding): Embedding(23447, 256)
  (fc): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=23447, bias=True)
  )
)


# 3.训练模型

In [25]:
# 训练20个epoch
for epoch in range(20):
    for train_input, train_label in tqdm(dataloader):
        output = model(train_input)
        loss = criterion(output, train_label.squeeze_())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, 'loss =', '{:.6f}'.format(loss))

  0%|          | 0/350 [00:00<?, ?it/s]

epoch: 1 loss = 8.726168


  0%|          | 0/350 [00:00<?, ?it/s]

epoch: 2 loss = 6.425567


  0%|          | 0/350 [00:00<?, ?it/s]

...

epoch: 18 loss = 0.008359


  0%|          | 0/350 [00:00<?, ?it/s]

epoch: 19 loss = 0.059392


  0%|          | 0/350 [00:00<?, ?it/s]

epoch: 20 loss = 0.006141


# 4.预测

In [26]:
# 使用训练好的模型进行预测
# 模型输出之后取argmax，再用idx2token转回单词，查看效果，预测结果有上下文关系
predict = model(train_input).data.max(1, keepdim=True)[1].squeeze_().tolist()
input_list = train_input.tolist()
for i in range(len(input_list)):
    print(dataset.vocab.get_itos()[input_list[i][0]] + ' ' +  
          dataset.vocab.get_itos()[input_list[i][1]] + ' ' + 
          dataset.vocab.get_itos()[input_list[i][2]] + ' -> ' + 
          dataset.vocab.get_itos()[predict[i]])

钲 设计 婚纱 -> 签
美国 男子 冒充 -> 警察
势 公益 进行 -> 到底
油价 后市 仍存 -> 下跌
子弟学校 千名 学生 -> 断电
王祖贤 恋情 多次 -> 放
股指 冲高 招行 -> 领涨
爆笑 不止 矢口否认 -> 传闻
员工 电脑公司 索赔 -> 亿美元
裁判 末位 淘汰制 -> 赛后
大行予 国泰 目标价 -> 至元
湖北 男子 找回 -> 失踪
中东 政策 演讲 -> 回应
著名 教练 孙 -> 广林
坦顿 詹姆斯 带来 -> 经验
名城 执法 遭 -> 小贩
万名 志愿者 监督 -> 网络
全 互联网 手机 -> 摩托罗拉
签 年长 约 -> 违约金
一季度 成本 压力 -> 继续
大学 年 艺术类 -> 专业
料全 年度 业绩 -> 录
低功耗 系列 惠普 -> 售元
日常 简单 七件 -> 事
忠 潜入 家中 -> 刺死
北京 陕西 穗 -> 京津
独显 屏 联想 -> 本降
高职 征求 志愿 -> 资格
朝鲜 拟 发射 -> 约
武汉 挑战赛 享受 -> 比赛
纷飞 文艺 女 -> 青年
曼城 米兰 大将 -> 出价
中招 入围 分数线 -> 划定
双卡 双待 纽曼 -> 直板
公牛 开拓者 擒灰熊 -> 轻取
装载 蔗糖 货船 -> 沉没
报名 截止日 回复 -> 提出
朝鲜 输给 大雨 -> 输给
给予 瑞金 矿业 -> 买入
诺基亚 中国 通信业 -> 进行
游戏场 完全 交战 -> 记录
认定 贪污受贿 万 -> 家属
专用 龙头 新品 -> 上市
宋佳获 最具 个性 -> 魅力
垃圾邮件 数量 再次 -> 回升
胜桑普 告捷 热那亚 -> 客场
年 地球 热销 -> 款
深基指 失守 创新 -> 低
借贷 万亿 金管局 -> 警示
猪梦 三国 娱乐 -> 杯
亚锦赛 中国 日本 -> 朱芳雨
三种 单身 风格 -> 组图
市场 熔盛 重工 -> 选择
艾弗森 挑错 保镖 -> 惹
市 方案 提振 -> 全球
学术 型 研究生 -> 减招
传奇 神秘 楼王 -> 引发
加冕 世界 女子 -> 短道
地产 蓄势 反攻 -> 量
莲花路 站 中午 -> 突发
午 盘道 指月 -> 首破
巴勒莫 官方 宣布 -> 夺下
率队 创亚 预赛 -> 最差
不死用 王一梅 封死 -> 韩
宣传 薛凯琪 三 -> 可怕
